<a href="https://colab.research.google.com/github/seirah-yang/BootCamp/blob/main/post_tester(%EB%AC%B8%EC%84%9C%EC%83%9D%EC%84%B1%EA%B8%B0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.6 MB/s eta 0:00:00


In [3]:
"""
Doc Evaluator (Refactored)
- 문서별 Claim 자동추출 (숫자·단위 KPI, 비율, 마감/기간 등 룰 기반)
- 하이브리드 검색: BM25 + 임베딩(BGE-M3). 임베딩/라이브러리 없으면 BM25로 폴백
- NLI + Boolean QA 결합 판정 (임계값/우선순위 적용)
- Major/Minor 지표 '실제' 집계 (고정값/더미 제거)
- 한국어 문장 분할 개선, heading 기반 섹션 추출, 형식 점검 개선
"""

import os, re, json, math
from typing import List, Dict, Any, Tuple
from collections import namedtuple, Counter
import docx

In [4]:
_EMBED_OK = False
_BM25_OK = False
try:
    from sentence_transformers import SentenceTransformer
    import numpy as np
    _EMBED_OK = True
except Exception:
    _EMBED_OK = False

try:
    from rank_bm25 import BM25Okapi
    _BM25_OK = True
except Exception:
    _BM25_OK = False

In [5]:
# 파서 & 전처리
class DocParser:
    def parse(self, docx_path: str):
        try:
            d = docx.Document(docx_path)
        except Exception:
            return None

        paras = [p.text.strip() for p in d.paragraphs if p.text and p.text.strip()]

        # Heading 기반 섹션 추출
        sections = []
        cur_title = None
        cur_buf = []
        for p in d.paragraphs:
            style = getattr(p.style, "name", "") or ""
            text = p.text.strip()
            if not text:
                continue
            if style.startswith("Heading") or "제목" in style:
                # 섹션 마감
                if cur_title or cur_buf:
                    sections.append({"title": cur_title, "text": "\n".join(cur_buf)})
                cur_title = text
                cur_buf = []
            else:
                cur_buf.append(text)
        if cur_title or cur_buf:
            sections.append({"title": cur_title, "text": "\n".join(cur_buf)})

        full_text = "\n".join(paras)
        sentences = split_ko_sentences(full_text)

        Doc = namedtuple("Doc", ["sections", "paragraphs", "sentences", "text"])
        return Doc(sections=sections, paragraphs=paras, sentences=sentences, text=full_text)

In [6]:
def split_ko_sentences(text: str) -> List[str]:
    # 문장 종결부호 + '다.' 패턴 반영
    # 공백/줄바꿈 기준으로 재조합
    text = re.sub(r"\s+", " ", text)
    sents = re.split(r"(?<=[\.?!])\s+|(?<=다\.)\s+", text)
    sents = [s.strip() for s in sents if s and s.strip()]
    return sents


In [7]:
# Claim(주장) 추출

KPI_PATTERNS = [
    r"\b\d{1,3}\s?%(\s?(감소|증가|유지))",     # 20% 감소/증가/유지
    r"(오류율|에러율)\s*\d{1,3}\s?%",         # 오류율 20%
    r"(정확도|완전성|재현율|정밀도)\s*(\d{1,3}\s?%)",  # 정확도 95%
    r"(응답시간|지연)\s*\d+(\.\d+)?\s?(ms|초)",       # 응답시간 500ms
    r"(처리량|TPS|QPS)\s*\d+(\.\d+)?",               # TPS 1000
    r"(기간|마감|데드라인)\s*(\d+\s?(일|주|개월|월|분기|년))",  # 기간 6개월
    r"(비용|원가)\s*\d+(,\d{3})*(\.\d+)?\s?(원|만원|억)",        # 비용 1,000만원
]

def extract_claims(doc) -> List[Dict[str, Any]]:
    claims = []
    for sent in doc.sentences:
        hit = False
        for pat in KPI_PATTERNS:
            if re.search(pat, sent):
                hit = True
                break
        if hit:
            claims.append({"sent": sent, "type": "KPI"})
    # 너무 적으면 상위 문장(제목/요약 추정)에서도 보완 수집
    if len(claims) < 5:
        head_boost = doc.paragraphs[:10]
        for s in head_boost:
            if any(re.search(p, s) for p in KPI_PATTERNS):
                claims.append({"sent": s, "type": "KPI"})
    # 중복 제거
    uniq = []
    seen = set()
    for c in claims:
        if c["sent"] not in seen:
            uniq.append(c); seen.add(c["sent"])
    return uniq[:15] if uniq else []

In [8]:
# 하이브리드 검색

class HybridRetriever:
    def __init__(self, cfg: Dict[str, Any], corpus_texts: List[str]):
        self.cfg = cfg
        self.corpus = corpus_texts or []
        self.use_embed = False
        self.use_bm25  = False

        # BM25
        if _BM25_OK and self.corpus:
            tokenized = [self.tokenize(x) for x in self.corpus]
            self.bm25 = BM25Okapi(tokenized)
            self.use_bm25 = True

        # 임베딩
        self.embed_dim = None
        if _EMBED_OK and self.corpus and cfg["models"].get("embed"):
            try:
                self.embed = SentenceTransformer(cfg["models"]["embed"])
                self.corpus_vec = self.embed.encode(self.corpus, normalize_embeddings=True)
                self.embed_dim = self.corpus_vec.shape[1]
                self.use_embed = True
            except Exception:
                self.use_embed = False

    @staticmethod
    def tokenize(x: str) -> List[str]:
        return re.findall(r"[가-힣A-Za-z0-9]+", x.lower())

    def topk(self, query: str, k: int = 5) -> List[Tuple[str, float]]:
        cand: Dict[int, float] = {}

        # BM25 스코어
        if self.use_bm25:
            scores = self.bm25.get_scores(self.tokenize(query))
            for i, s in enumerate(scores):
                if s > 0:
                    cand[i] = cand.get(i, 0.0) + float(s)

        # 임베딩 스코어(코사인)
        if self.use_embed:
            qv = self.embed.encode([query], normalize_embeddings=True)[0]
            sims = (self.corpus_vec @ qv)
            for i, s in enumerate(sims):
                cand[i] = cand.get(i, 0.0) + float(s) * 100.0  # 임베딩 가중(스케일 정규화 목적)

        # 둘 다 실패한 경우: 부분문자열 폴백
        if not cand:
            for i, t in enumerate(self.corpus):
                if query[:20] in t:
                    cand[i] = 1.0

        ranked = sorted(cand.items(), key=lambda x: x[1], reverse=True)[:k]
        return [(self.corpus[i], score) for i, score in ranked]


In [9]:
# NLI / QA (더미+규칙 강화)

class NLIModel:
    """
    외부 대형 NLI 모델이 없는 환경에서도 동작하도록 규칙 기반 점수 제공.
    - 숫자/단위 일치도가 높으면 entailment를, 상충 수치가 발견되면 contradiction을 강화
    """
    def __init__(self, cfg):
        self.cfg = cfg

    def predict(self, claim: str, evidence: str) -> Dict[str, float]:
        claim_nums = extract_numbers_units(claim)
        evid_nums  = extract_numbers_units(evidence)

        # 기본 확률
        entail = 0.33; contra = 0.33; neutr = 0.34
        match_cnt, conflict = number_match_quality(claim_nums, evid_nums)

        if match_cnt > 0 and not conflict:
            entail = 0.7; contra = 0.1; neutr = 0.2
        elif conflict:
            entail = 0.1; contra = 0.7; neutr = 0.2
        else:
            # 유사 텍스트 힌트
            if keyword_overlap(claim, evidence) > 0.4:
                entail = 0.5; neutr = 0.4; contra = 0.1

        max_p = max(entail, contra, neutr)
        return {"entail": entail, "contra": contra, "neutral": neutr, "max_p": max_p}


class BooleanQA:
    """아주 짧은 yes/no 룰베이스. (실모델 없을 때 폴백)"""
    def __init__(self, cfg):
        self.cfg = cfg

    def yesno(self, question: str, context: str) -> str:
        # 키워드 중첩/수치 매칭이 일정 이상이면 yes
        q = question.lower()
        ko = keyword_overlap(q, context)
        cn, en = extract_numbers_units(q), extract_numbers_units(context)
        match_cnt, conflict = number_match_quality(cn, en)
        if conflict:
            return "no"
        if match_cnt >= 1 or ko > 0.35:
            return "yes"
        return "no"


In [10]:
# 평가 지표 유틸

def extract_numbers_units(text: str) -> List[Tuple[float, str]]:
    out = []
    # 20%, 95 %, 500ms, 10개월, 1,000만원 등
    for m in re.finditer(r"(\d+(?:[\.,]\d+)?)(\s?%|ms|초|일|주|개월|월|분기|년|원|만원|억)?", text):
        num = m.group(1).replace(",", "")
        unit = (m.group(2) or "").strip()
        try:
            out.append((float(num), unit))
        except Exception:
            pass
    return out

def number_match_quality(a: List[Tuple[float,str]], b: List[Tuple[float,str]]) -> Tuple[int, bool]:
    """같은 단위에서 숫자가 근접하면 match, 큰 차이면 conflict로."""
    match = 0; conflict = False
    for ax, au in a:
        for bx, bu in b:
            if au and bu and au == bu:
                # 근사 일치(상대 오차 10% 이내)
                if bx == 0:
                    continue
                rerr = abs(ax - bx) / (abs(bx) + 1e-6)
                if rerr <= 0.1:
                    match += 1
                elif rerr >= 0.5:
                    conflict = True
    return match, conflict

def keyword_overlap(a: str, b: str) -> float:
    ta = set(re.findall(r"[가-힣A-Za-z0-9]+", a.lower()))
    tb = set(re.findall(r"[가-힣A-Za-z0-9]+", b.lower()))
    if not ta or not tb:
        return 0.0
    return len(ta & tb) / len(ta | tb)

def ngram_redundancy(sentences: List[str], n: int = 3) -> float:
    """3-gram 중복 비율"""
    grams = []
    for s in sentences:
        toks = re.findall(r"[가-힣A-Za-z0-9]+", s.lower())
        grams += list(zip(*[toks[i:] for i in range(n)]))
    if not grams:
        return 0.0
    c = Counter(grams)
    dup = sum(v-1 for v in c.values() if v>1)
    return dup / (len(grams) + 1e-6)

def simple_coherence(sentences: List[str]) -> float:
    """아주 단순한 연결성: 인접 문장 키워드 교집합 비율 평균"""
    if len(sentences) < 2:
        return 0.5
    scores = []
    for i in range(len(sentences)-1):
        scores.append(keyword_overlap(sentences[i], sentences[i+1]))
    return sum(scores)/len(scores)

def format_score(sections: List[Dict[str,str]], required_titles: List[str]) -> float:
    titles = [ (s["title"] or "").strip() for s in sections if s.get("title") ]
    hit = 0
    for req in required_titles:
        # 부분 일치 허용
        if any(req in (t or "") for t in titles):
            hit += 1
    if not required_titles:
        return 1.0
    return hit / len(required_titles)

def aggregate_scores(major: Dict[str, float], minor: Dict[str, float], glossary_on: bool) -> Dict[str, float]:
    # terminology는 glossary 있을 때만 반영
    term_w = 0.1 if glossary_on else 0.0
    norm_w = 1.0 - term_w
    # minor 합성
    minor_core = (
        0.4 * minor["coherence"] +
        0.3 * minor["fluency"] +
        0.2 * (1 - minor["redundancy"]) +
        0.1 * minor["format"]
    )
    minor_total = norm_w * minor_core + term_w * minor.get("terminology", 0.0)
    final = 0.6 * major["accuracy"] + 0.4 * minor_total
    return {"final_score": final}

In [11]:
# 리포트
def build_report(doc, results, major, minor, final, claimed_top=10) -> str:
    rep = []
    rep.append("## 문서 평가 보고서\n")
    rep.append(f"### 최종 점수: {final['final_score']:.3f}\n")
    rep.append("### 주요 오류 분석 (Major)\n")
    rep.append(f"- Accuracy: {major['accuracy']:.3f} (entail={major['entail_cnt']}, contra={major['contra_cnt']}, unknown={major['unknown_cnt']})\n")
    if "precision" in major:
        rep.append(f"- Precision: {major['precision']:.3f}, Recall: {major['recall']:.3f}, F1: {major['f1']:.3f}\n")
    rep.append("\n#### 개별 주장 검증 상위 결과\n")
    for r in results[:claimed_top]:
        rep.append(f"- 주장: {r['sent']}\n")
        best_txt = r['best_evidence'][:300].replace("\n", " ")
        rep.append(f"  - 근거: {best_txt} ...\n")
        rep.append(f"  - 판정: {r['verdict']} (신뢰도: {r['confidence']:.2f})\n\n")

    rep.append("### 사소 오류 분석 (Minor)\n")
    rep.append(f"- Fluency: {minor['fluency']:.3f}\n")
    rep.append(f"- Coherence: {minor['coherence']:.3f}\n")
    rep.append(f"- Redundancy(낮을수록 좋음): {minor['redundancy']:.3f}\n")
    rep.append(f"- Format: {minor['format']:.3f}\n")
    if "terminology" in minor:
        rep.append(f"- Terminology: {minor['terminology']:.3f}\n")

    # 개선 제안
    rep.append("\n### 개선 제안\n")
    if major["contra_cnt"] > 0 or major["unknown_cnt"] > 0:
        rep.append("- KPI에 대한 정확한 수치/근거 인용을 본문/부록에 명시하세요(표·각주·참고문헌 라벨).\n")
    if minor["redundancy"] > 0.25:
        rep.append("- 중복 문장을 통합하세요(요약/표로 치환, 불필요한 반복 제거).\n")
    if minor["format"] < 0.8:
        rep.append("- 필수 섹션(연구개발 목표/내용/기대효과/추진체계/활용계획 등)의 제목을 명시하고 순서를 정리하세요.\n")
    rep.append("- 섹션 간 연결어(따라서, 한편, 결과적으로 등)를 활용해 응집성을 높이세요.\n")

    return "".join(rep)

In [31]:
# 평가기
# ======================
class DocEvaluator:
    def __init__(self, cfg: Dict[str, Any]):
        self.cfg = cfg
        self.parser = DocParser()
        self.nli = NLIModel(cfg["models"].get("nli"))
        self.boolqa = BooleanQA(cfg["models"].get("qna"))
        # 용어집
        self.glossary = self._load_glossary(cfg["inputs"].get("domain_glossary"))

    def _load_glossary(self, file_path: str):
        if not file_path or not os.path.exists(file_path):
            return None
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                return json.load(f)
        except Exception:
            return None

    def evaluate(self, docx_path: str) -> Dict[str, Any]:
        doc = self.parser.parse(docx_path)
        if not doc:
            return {"final_score": 0.0, "error": "Document parsing failed"}

        # 코퍼스(근거 후보): 문장 + 섹션 텍스트
        corpus = doc.sentences[:]
        for s in doc.sections:
            corpus.append(s["text"])
        retriever = HybridRetriever(self.cfg, corpus)

        # 1) Claim 추출
        claims = extract_claims(doc)
        if not claims:
            # 주장 없으면 문서 핵심문장(상위 10개)로 대체
            base = [{"sent": s, "type": "fallback"} for s in doc.sentences[:10]]
            claims = base

        # 2) 검증
        results = []
        entail_cnt = contra_cnt = unknown_cnt = 0
        for c in claims:
            top = retriever.topk(c["sent"], k=5)
            evidences = [t for t, sc in top]
            best = evidences[0] if evidences else ""
            nli = self.nli.predict(c["sent"], best)
            verdict, conf = self._post_decide(c["sent"], best, nli)

            if verdict == "entailment": entail_cnt += 1
            elif verdict == "contradiction": contra_cnt += 1
            else: unknown_cnt += 1

            results.append({
                **c,
                "best_evidence": best,
                "verdict": verdict,
                "confidence": conf
            })

             # 3) Major 지표
        tot = max(1, (entail_cnt + contra_cnt + unknown_cnt))
        accuracy = entail_cnt / tot
        # QA 기반 precision/recall 추정(간단 정의): yes=정답, no=오답 가정
        tp = entail_cnt
        fp = contra_cnt
        fn = unknown_cnt
        precision = tp / (tp + fp + 1e-6)
        recall    = tp / (tp + fn + 1e-6)
        f1        = 2*precision*recall / (precision+recall+1e-6)
        major_scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "entail_cnt": entail_cnt,
            "contra_cnt": contra_cnt,
            "unknown_cnt": unknown_cnt
        }
 # 3) Major 지표
        tot = max(1, (entail_cnt + contra_cnt + unknown_cnt))
        accuracy = entail_cnt / tot
        # QA 기반 precision/recall 추정(간단 정의): yes=정답, no=오답 가정
        tp = entail_cnt
        fp = contra_cnt
        fn = unknown_cnt
        precision = tp / (tp + fp + 1e-6)
        recall    = tp / (tp + fn + 1e-6)
        f1        = 2*precision*recall / (precision+recall+1e-6)
        major_scores = {
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "entail_cnt": entail_cnt,
            "contra_cnt": contra_cnt,
            "unknown_cnt": unknown_cnt
        }
    def _post_decide(self, clain: str, evidence: str, nli_out: Dict[str, float]) -> Tuple[str, float]:
         # 임계값
         conf = nli_out["max_p"]
         if conf >= 0.65:
            return self.argmax_verdic(nli_out), conf

            qa_ans = self.boolqa.yesno(f"Is the claim supported? {claim}", evidence)
            if qa_ans.lower().startswith("y"):
                return "entailment", max(0.65, conf)
            else:
                return "contradiction", max(0.65, conf)

    @staticmethod
    def _argmax_verdict(nli_out: Dict[str,float]) -> str:
        m = max((("entailment", nli_out["entail"]),
                ("contradiction", nli_out["contra"]),
                ("neutral", nli_out["neutral"])), key=lambda x:x[1])[0]
        return m

    def _fluency(self, sents: List[str]) -> float:
        # 간이 유창성: 평균 문장 길이·기호 비율을 바탕으로 0~1 스코어
        if not sents:
            return 0.5

        lens = [len(s) for s in sents]
        mean_len = sum(lens)/len(lens)
        punct = sum(ch in ".,;:?!~" for s in sents for ch in s) / (sum(lens)+1e-6)

        # 경험적 매핑
        score = 0.5 + 0.5 * math.tanh((mean_len-25)/50) - 0.2*abs(punct-0.03)
        return max(0.0, min(1.0, score))

        def _terminology_score(self, text: str):
            if not self.glossary:
                return None
            terms = set(self.glossary.get("terms", []))
            if not terms:
                return None
            toks = set(re.findall(r"[가-힣A-Za-z0-9\-]+", text))
            hit = len(terms & toks)
            return hit / len(terms)


In [32]:
if __name__ == "__main__":
    config = {
        "models": {
            "embed": "BAAI/bge-m3",          # 있으면 사용, 없으면 자동 폴백
            "nli":  "rule-lite",
            "qna":  "rule-lite"
        },
        "inputs": {
            "domain_glossary": None
        },
        "format_required": [
            "연구개발 목표", "연구개발 내용", "연구개발성과 활용계획", "추진체계", "기대효과"
        ],
        "output_dir": "./eval_reports"
    }

In [ ]:
 import argparse
    ap = argparse.ArgumentParser()
    ap.add_argument("--doc", required=True, help="/content/e5_연구계발계획서v6.docx")
    args = ap.parse_args()

    ev = DocEvaluator(config)
    out = ev.evaluate(args.doc)
    print(f"[DONE] score={out['final_score']:.3f}, report={out['report_path']}")

In [ ]:
 import argparse
    ap = argparse.ArgumentParser()
    ap.add_argument("--doc", required=True, help="/content/cde_연구계발계획서.docx")
    args = ap.parse_args()

    ev = DocEvaluator(config)
    out = ev.evaluate(args.doc)
    print(f"[DONE] score={out['final_score']:.3f}, report={out['report_path']}")